In [1]:
from scipy import sparse
import pickle
import argparse
import logging
import pandas as pd
import numpy as np
import anndata

from robustlink import utils
from robustlink import enhancer_gene_utils

/Users/fangmingxie/opt/anaconda3/envs/robustlink3/lib/python3.8/site-packages/anndata/core/anndata.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version. The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [12]:
mod_x = 'rna'
mod_y = 'mc'
input_name_tag = ''

modx = mod_x
mody = mod_y
ka = 30
knn = 30
date = 211115
input_name_tag=f"mop_{modx}_{mody}_ka{ka}_knn{knn}_{date}" # 
i_sub = 0

In [13]:
# input enh-gene tables, gene-by-cell, enhancer-by-cell matrices
input_enh_gene_table = './data/counts/enhancer_gene_pairs_1mbp.tsv' 
input_bundle_dirc = './data/counts'
bundle_fnames = (
    'cell_rna.pkl',
    'cell_mc.pkl',

    'gene_rna.pkl',
    'enh.pkl',

    'mat_rna.pkl',
    'mat_mcg_mc.pkl',
    'mat_cg_mc.pkl',
)

# for knn_xx
input_knn_dirc = './results'
input_modx_clsts = [
    'clusterings_{}_{}_sub{}.tsv.gz'.format(mod_x, input_name_tag, i_sub),
]

# for knn_xy
input_knn_xy = 'knn_across_{}_{}_{}.npz.{}.npz'.format(input_name_tag, mod_x, mod_y, i_sub) 
input_knn_cells_xaxis = 'cells_{}_{}.npy.{}.npy'.format(mod_x, input_name_tag, i_sub)
input_knn_cells_yaxis = 'cells_{}_{}.npy.{}.npy'.format(mod_y, input_name_tag, i_sub)

In [14]:
# input_bundle
with utils.cd(input_bundle_dirc):
    bundle = []
    for fname in bundle_fnames:
        #  save all as pickle file
        with open(fname, "rb") as fh:
            item = pickle.load(fh)
        bundle.append(item)
        logging.info("{}_{}_{}".format(type(item), item.shape, fname))

(common_rna_cells, common_mc_cells, 
 common_genes, 
 common_enhancer_regions,
 X, Y_mcg, Y_cg, 
#  knn_xy, knn_xx,
) = bundle

/var/folders/5t/5ql_465s2w7g1s3t2c4ttz7c0000gp/T/ipykernel_79041/29806297.py:7: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  item = pickle.load(fh)

KeyboardInterrupt



In [5]:
rna_data = anndata.AnnData(
    X=X, 
    obs=pd.DataFrame(index=common_genes.astype(str)),
    var=pd.DataFrame(index=common_rna_cells.astype(str)),
)
rna_data

AnnData object with n_obs × n_vars = 31053 × 69727 

In [6]:
rna_data.write('./newdata/gene_rna_counts.h5ad')

In [7]:
enh_data = anndata.AnnData(
                X=None,
                obs=common_enhancer_regions.astype({'chr': '|S'}),
                var=pd.DataFrame(index=common_mc_cells.astype(str)),
                )
enh_data.layers['mcg'] = Y_mcg
enh_data.layers['cg'] = Y_cg
enh_data

Transforming to str index.


AnnData object with n_obs × n_vars = 233514 × 9931 
    obs: 'chr', 'start', 'end'
    layers: 'mcg', 'cg'

In [8]:
enh_data.write('./newdata/enh_mc_counts.h5ad')

/Users/fangmingxie/opt/anaconda3/envs/robustlink3/lib/python3.8/site-packages/anndata/core/anndata.py:1328: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  and not is_categorical(df[key])
... storing 'chr' as categorical
/Users/fangmingxie/opt/anaconda3/envs/robustlink3/lib/python3.8/site-packages/anndata/core/anndata.py:108: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  elif is_categorical(df[k]):


# check the data is alright

# save knn_xy as AnnData

In [15]:
# input knn networks 
with utils.cd(input_knn_dirc):
    # for knn_xx 
    # modx_clsts = pd.read_csv(input_modx_clsts, sep='\t',index_col=0)
    modx_clsts = pd.concat([
        pd.read_csv(fname, sep='\t',index_col=0)
        for fname in input_modx_clsts
    ], axis=1)
    # for knn_xy 
    knn_xy = sparse.load_npz(input_knn_xy)  
    cell_cell_knn_xaxis = np.load(input_knn_cells_xaxis, allow_pickle=True)
    cell_cell_knn_yaxis = np.load(input_knn_cells_yaxis, allow_pickle=True)

    logging.info("{} {} {} {}".format(
          modx_clsts.shape, 
          knn_xy.shape, 
          cell_cell_knn_xaxis.shape, 
          cell_cell_knn_yaxis.shape,
          )
         )

# enhancer-gene linkage
enhancer_gene_to_eval = pd.read_csv(input_enh_gene_table, sep='\t')

In [29]:
aknn_xy = anndata.AnnData(X=knn_xy.tocsr(), # csr or csc?
                obs=pd.DataFrame(index=cell_cell_knn_xaxis.astype(str)),
                var=pd.DataFrame(index=cell_cell_knn_yaxis.astype(str)),
                )

In [30]:
aknn_xy

AnnData object with n_obs × n_vars = 6898 × 963 

In [31]:
aknn_xy.write('./newdata/knn_xy.h5ad')

# do the same for ATAC